In [0]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
import sparknlp
spark=sparknlp.start()
sc=pyspark.SparkContext.getOrCreate()

In [0]:
dataframe=spark.read.csv("/home/cse587/Desktop/data/train.csv",inferSchema=True,header=True,escape='"')
dataframe.printSchema()


root
 |-- movie_id: string (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import (StopWordsRemover,HashingTF, Tokenizer,CountVectorizer)
import numpy as np

from pyspark.ml import Pipeline

In [0]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
countTokens = udf(lambda words: len(words), IntegerType())
regexTokenized = regexTokenizer.transform(dataframe)
regexTokenized.select("words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
stopwordsremoved=remover.transform(regexTokenized)
stopwordsremoved.select('filtered').show(truncate=False)



+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import regexp_extract, when

mapping_csv=spark.read.load('/home/cse587/Desktop/data/mapping.csv',format="csv",sep=',',inferSchema='true',header='true',escape='"')
stopwordsremoved.select('genre').show(truncate=False)
col1=mapping_csv.select('0').collect()
list1=[str(i[0]) for i in col1]   
for val in list1:
    stopwordsremoved = stopwordsremoved.withColumn(val, when(stopwordsremoved['genre'].contains("'"+ val +"'"),1.0).otherwise(0.0))

stopwordsremoved.show()


+-------------------------------------------------------------------------+
|genre                                                                    |
+-------------------------------------------------------------------------+
|['World cinema', 'Drama']                                                |
|['Action/Adventure', 'Action', 'Science Fiction', 'Drama']               |
|['Musical', 'Action', 'Drama']                                           |
|['Comedy']                                                               |
|['Crime Fiction', 'World cinema', 'Drama']                               |
|['Action/Adventure', 'Action', 'Thriller', 'Drama']                      |
|['Thriller', 'Drama', 'Horror']                                          |
|['Drama']                                                                |
|['Black-and-white', 'Comedy', 'Romance Film', 'Romantic comedy', 'Drama']|
|['Animation', 'Short Film', 'Family Film']                               |
|['Comedy'] 

In [0]:
test_csv=spark.read.load('/home/cse587/Desktop/data/test.csv',format="csv",sep=',',inferSchema='true',header='true',escape='"')


test_csv.printSchema()
regexTokenizer_test = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
countTokens = udf(lambda words: len(words), IntegerType())
regexTokenized_test = regexTokenizer_test.transform(test_csv)
regexTokenized_test.select("words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

remover_test = StopWordsRemover(inputCol="words", outputCol="filtered")
stopwordsremoved_test=remover_test.transform(regexTokenized_test)
stopwordsremoved_test.select('filtered').show(truncate=False)
test_csv=stopwordsremoved_test

test_csv.show()

root
 |-- movie_id: integer (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
trainingData=stopwordsremoved
trainingData.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Drama: double (nullable = false)
 |-- Comedy: double (nullable = false)
 |-- Romance Film: double (nullable = false)
 |-- Thriller: double (nullable = false)
 |-- Action: double (nullable = false)
 |-- World cinema: double (nullable = false)
 |-- Crime Fiction: double (nullable = false)
 |-- Horror: double (nullable = false)
 |-- Black-and-white: double (nullable = false)
 |-- Indie: double (nullable = false)
 |-- Action/Adventure: double (nullable = false)
 |-- Adventure: double (nullable = false)
 |-- Family Film: double (nullable = false)
 |-- Short Film: double (nullable = false)
 |-- Romantic drama: double (nullable = false)
 |-- Animation: double (nu

**P2 using TF-IDF**

In [0]:
from pyspark.ml.classification import LinearSVC,NaiveBayes
from pyspark.ml.feature import HashingTF, IDF

hashingTF=HashingTF(inputCol = "filtered", outputCol='Features_org') 
htf_trainingData = hashingTF.transform(trainingData)
htf_testData = hashingTF.transform(test_csv)

idf_model=IDF(inputCol = 'Features_org' , outputCol = 'features')
idf = idf_model.fit(htf_trainingData)

idf_trainingData = idf.transform(htf_trainingData)
idf_testData = idf.transform(htf_testData) 


In [0]:
test_result=idf_testData
test_result.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               words|            filtered|        Features_org|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|(262144,[1728,261...|(262144,[1728,261...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|(262144,[6068,191...|(262144,[6068,191...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|(262144,[1598,208...|(262144,[1598,208...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|(262144,[3294,618...|(262144,[3294,618...|
|24165951|        Getting Even|A soldier-

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.util import MLUtils
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.mllib.classification import LogisticRegressionModel, LogisticRegressionWithLBFGS
import pickle
#test_result = test_result.withColumn("id",monotonically_increasing_id())
import os, tempfile
count=0
for val in list1:
    print(val)
    trainingData_val=idf_trainingData.withColumnRenamed(val,'label')
    input_train = MLUtils.convertVectorColumnsFromML(trainingData_val.select('label','features')).rdd
    input_train = input_train.map(lambda row : LabeledPoint(row.label, row.features))
    # lrm = LogisticRegressionWithLBFGS.train(input_train)

    # with open("/content/drive/My Drive/dic487-587/P2-Main-models/"+str(count),"wb")as fid: pickle.dump(lrm,fid)
    # lrm.save(sc,"/content/drive/My Drive/"+val)

    
    loadedmodel=pickle.load(open("/home/cse587/Desktop/Assign3/Part2/LR_model/"+str(count),"rb"))
    input_test = MLUtils.convertVectorColumnsFromML(idf_testData.select('movie_id','features')).rdd
    testData_pred=input_test.map(lambda a: (a.movie_id, loadedmodel.predict(a.features)))
    sqlContext = SQLContext(sc)
    testData_val = sqlContext.createDataFrame(testData_pred)
    testData_val.createOrReplaceTempView("testData_pred")
    testData_val=testData_val.withColumnRenamed('_1','movie_id')
    testData_val=testData_val.withColumnRenamed('_2', val)
    test_result = test_result.join(testData_val, test_result.movie_id == testData_val.movie_id).select(test_result["*"],testData_val[val])
    count=count+1
#test_result.show()


Drama
Comedy
Romance Film
Thriller
Action
World cinema
Crime Fiction
Horror
Black-and-white
Indie
Action/Adventure
Adventure
Family Film
Short Film
Romantic drama
Animation
Musical
Science Fiction
Mystery
Romantic comedy
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+---------------+
|movie_id|          movie_name|                plot|               words|            filtered|        Features_org|            features|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|Animation|Musical|Science Fiction|Mystery|Romantic comedy|
+--------+--------------------+--------------------+-----------

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import concat,lit
from pyspark.sql.types import StringType
test_result=test_result.withColumn("predictions",lit("").cast(StringType()))

for val in list1:
  test_result=test_result.withColumn("predictions",concat(col("predictions") ,lit(" "), col(val)))

In [0]:
exportfile=test_result.select("movie_id","predictions")
exportfile.show(truncate=False)

+--------+----------------------------------------+
|movie_id|predictions                             |
+--------+----------------------------------------+
|62693   | 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|296252  | 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0|
|1356971 | 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|1428872 | 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0|
|1582173 | 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0|
|1595142 | 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|1600825 | 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0|
|1681132 | 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0|
|2268290 | 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 1|
|3327607 | 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|3569957 | 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|4413498 | 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0|
|4635580 | 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|4950989 | 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0|
|5565692 | 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|7003785 | 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0|
|8269287 | 1

In [0]:
exportfile.coalesce(1) \
       .write \
       .csv("/home/cse587/Desktop/Part2_submission",header=True)